# Sedimentation

In [ ]:
from trustutils.jupyter import run

run.introduction("A. Gerschenfeld, Y. Gorsse")

run.TRUST_parameters()

### Description: 

Cas tests de sedimentation. Etat initial melange, et separation des phases par gravite.

In [ ]:
import os
from string import Template
run.useMEDCoupling()
import medcoupling as mc

meshes_dir = os.environ["TRUST_ROOT"] + "/Validation/Rapports_automatiques/Verification/PolyMAC/share/meshes"

cases = {("Cartesian", 4) : ["PolyMAC_P0", "VDF"], ("Triangles", 3) : ["PolyMAC_P0"], ("Quadrangles", 3) : ["PolyMAC_P0"], ("Locally_Refined", 3) : ["PolyMAC_P0"]}
virtual_mass = { "no_vm" : " ", "vm" : " masse_ajoutee coef_constant { beta 0.5 } " }
run.reset()
run.initBuildDirectory()

for (case, n_mesh), diss in cases.items():
    for dis in diss:
        for (vm, string_vm) in virtual_mass.items() :
            os.system(f"mkdir -p {run.BUILD_DIRECTORY}/{case}/{dis}/{vm}")
            with open(f"{run.BUILD_DIRECTORY}/sep.data", "r") as file: filedata = Template(file.read())
            result = filedata.substitute({"dis" : dis, "virtual_mass" : string_vm})
            with open(f"{run.BUILD_DIRECTORY}/{case}/{dis}/{vm}/jdd.data", "w") as file: file.write(result)
            if not dis.startswith("Poly"):
                os.system(f"gunzip -c {meshes_dir}/{case}/jdd_{n_mesh}/mesh.med.gz >{run.BUILD_DIRECTORY}/{case}/{dis}/{vm}/mesh.med")
                meshname = f"{run.BUILD_DIRECTORY}/{case}/{dis}/{vm}/mesh.med"
                mfum = mc.MEDFileUMesh(meshname)
                mfum.getMeshAtLevel(0).unPolyze()
                mfum.write(meshname, 2)
            else: os.system(f"gunzip -c {meshes_dir}/{case}/jdd_{n_mesh}/mesh.med.gz >{run.BUILD_DIRECTORY}/{case}/{dis}/{vm}/mesh.med")

            run.addCase(f"{case}/{dis}/{vm}" ,"jdd.data")

run.printCases()
run.runCases()
run.tablePerf()

## Resultats

In [ ]:
from trustutils import plot

times = [1.0, 2.0, 4.0, 7.0, 10.0]

for (case, _), diss in cases.items():
    for dis in diss:
        for vm in virtual_mass.keys() :
            a = plot.Graph(f"{case} - {dis} - {vm}", nY=2)
            for t in times:
                a.addPlot(0, "Alpha")
                a.addSegment(f"{case}/{dis}/{vm}/jdd_ALPHA.son", time=t, label=f"t={t}s")
                a.label("y [m]","alpha [-]")
                a.visu(0,1,0,1)

                a.addPlot(1, "Vg")
                a.addSegment(f"{case}/{dis}/{vm}/jdd_PROFIL_VG.son", compo=1, time=t, label=f"t={t}s")
                a.label("y [m]","vg [m/s]")
                a.visu(0,1,-0.2,0.2)
